# Tarea 2
A partir del corpus proporcionado realizar un modelo del lenguaje neuronal con base en la arquitectura de word2vec. Siganse los siguientes pasos:

1. Limpiar los Textos y apliccar stemming a las palabras. **[DONE]**
2. Insertar simbolos de inicio y final de cadena. **[DONE]**
3. Obtener los bigramas que aparecen en este texto. 
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparamteros. Tomar de 100 a 300 unidades para la capa oculta.
5. Obtener la matriz A y Pi a partir de las salidas de la red neuronal 
6. Calcular la probabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente 
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

In [23]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos. En este caso definimos dos formas de cargar el corpus, ya sea por documento o cargando todos los documentos del folder.

In [24]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"/CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        

In [25]:
# Cargamos el corpus.

#corpus = loadAllCorpus()
corpus = loadCorpus('corpusML.txt')

## Limpieza del Texto
Separamos las palabras de las oraciones para poder trabajar con ellas individualmente

In [26]:
def add_eos_init(corpus):
    """
    Adds <eos> and <init> to a corpus. Based on line jumps
    """
    
    init = '<init> '
    eos = '<eos>'
    corpus_in_eo = []


    corpus_in_eo = init + corpus + eos
    corpus_in_eo= corpus_in_eo.replace("\n", eos +" \n"+init)

    return corpus_in_eo

In [27]:
corpus_init = add_eos_init(corpus)
words = corpus_init.split()
print(words[:20])

['<init>', 'Comencé', 'a', 'trabajar', 'y', 'me', 'pegaron,', 'me', 'maltrataron', 'con', 'chicote', '<eos>', '<init>', 'Mis', 'patrones', 'me', 'pegaron', 'porque', 'no', 'me']


Eliminamos la puntuación del documento, acentos y normalizamos el texto en minusculas. Para hacer la eliminación de los símbolos de puntuación utilziamos una tabla de traducción para optimizar la velocidad de procesamiento. Tambien fue necesario extender la tabla de símbolos para que incluyera algunos símbolos latinos que faltaban.

Para eliminar acentos usamos la libreria unidecode que se tiene que instalar adicionalmente: `pip install unidecode`

In [28]:
import string
import unidecode

# Para poder mantener algunos flags quitamos < > de los elementos que se pueden eliminar
# de los simbolos de puntuación
punct = string.punctuation.replace("<", '')
punct = punct.replace(">", '')

print(punct)

!"#$%&'()*+,-./:;=?@[\]^_`{|}~


In [29]:
lat_punctuation = punct+'¿¡1234567890'

table = str.maketrans('', '', lat_punctuation)

In [30]:
clean_words = []

for word in words:
    word = word.translate(table)      # Quitamos simbolos de puntuacion
    word = word.lower()               # Minusculas
    word = unidecode.unidecode(word)  # Quitamos acentos.
    clean_words.append(word)

clean_words[:20]

['<init>',
 'comence',
 'a',
 'trabajar',
 'y',
 'me',
 'pegaron',
 'me',
 'maltrataron',
 'con',
 'chicote',
 '<eos>',
 '<init>',
 'mis',
 'patrones',
 'me',
 'pegaron',
 'porque',
 'no',
 'me']

## Stemming de Palabras
Para hacer el stemming de las palabras usamos NLTK. Para esto hay que instalar NLTK: 

Lo primero que hacemos es definir un stemmer. En este caso usaremos [Snowball Stemmer](http://snowball.tartarus.org/texts/introduction.html).

In [31]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [32]:
stemmed_text = []
for word in clean_words:
    stemmed_text.append(stemmer.stem(word))
    
print(stemmed_text[:10])

['<init>', 'comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con']


<init>Comencé a trabajar y me pegaron, me maltrataron con chicote <eos> 
<init>Mis patrones me pegaron porque no me quería apurar, porque era flojo <eos> 
<init>Por eso, me habían pegado <eos> 
<init>Cuando me pegaban ya entonces me quitaba <eos> 
<init>Pues entonces no quise trabajar <eos> 
<init>Ya no quise estar, como me pegaban <eos> 
<init>Después ya estuve nomás en mi casa <eos> 
<init>Fui a juntar caca de caballo <eos> 
<init>Fui a juntar caquitas de caballo <eos> 
<init>Se me aventaron las culebras <eos> 
<init>Se me aventaron los animales <eos> 
<init>Pero se me aventaban víboras muy grandes <eos> 
<init>Me asusté <eos> 
<init>Otra vez me quité <eos> 
<init>Me asusté con las víboras que se me aventaban, con los animales <eos> 
<init>Luego entonces otra vez me estuve nomás <eos> 
<init>Otra vez me estuve nomás, entré a la escuela pero no me hallé otra vez <eos> 
<init>No supe nada, pues me sacaron <eos> 
<init>Me sacó mi padre y mi madre <eos> 
<init>Luego entonces pues otra ve

In [27]:
print(corpus[-2:])

s



# Realizado por:
- Bustos Ramírez Luis Enrique
- Flores Cortés Juan Pablo
